# 05 • Consultas e Métricas

Objetivo: preparar queries e métricas que o app vai usar (e validar resultados).


In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import re, unicodedata

# Caminhos do projeto (ajuste se necessário)
BASE_DIR = Path.cwd().resolve().parents[0]  # .../notebooks -> projeto
RAW_DIR = BASE_DIR / "data" / "raw"
PROCESSED_DIR = BASE_DIR / "data" / "processed"
OUTPUTS_DIR = BASE_DIR / "outputs"

RAW_DIR, PROCESSED_DIR, OUTPUTS_DIR


(WindowsPath('C:/Users/User/Desktop/python/ali_rural_prospecção/data/raw'),
 WindowsPath('C:/Users/User/Desktop/python/ali_rural_prospecção/data/processed'),
 WindowsPath('C:/Users/User/Desktop/python/ali_rural_prospecção/outputs'))

In [2]:
import sqlite3

In [3]:
db_path = PROCESSED_DIR / "imoveis_rurais.sqlite"
db_path.exists(), db_path

(True,
 WindowsPath('C:/Users/User/Desktop/python/ali_rural_prospecção/data/processed/imoveis_rurais.sqlite'))

In [4]:
conn = sqlite3.connect(db_path)

df_mun = pd.read_sql_query("SELECT municipio, COUNT(*) as n FROM imoveis GROUP BY municipio ORDER BY n DESC;", conn)
df_cond = pd.read_sql_query("SELECT condicao_pessoa, COUNT(*) as n FROM imoveis GROUP BY condicao_pessoa ORDER BY n DESC;", conn)
df_nat = pd.read_sql_query("SELECT natureza_juridica, COUNT(*) as n FROM imoveis GROUP BY natureza_juridica ORDER BY n DESC LIMIT 50;", conn)

conn.close()

df_mun.head(10)

,municipio,n
0,BRAGANÇA PAULISTA,5194
1,JUNDIAÍ,4500
2,ITATIBA,2577
3,JARINU,2367
4,PINHALZINHO,1914
5,PEDRA BELA,1500
6,ITUPEVA,1317
7,CABREÚVA,1175
8,LOUVEIRA,1011
9,MORUNGABA,987


In [5]:
df_cond.head(15)

,condicao_pessoa,n
0,Proprietario Ou Posseiro Comum,12946
1,Proprietario Ou Posseiro Individual,11945
2,Nu-Proprietario,850
3,Usufrutuário,539
4,Proprietario Ou Posseiro Associado,52


In [6]:
df_nat.head(15)

,natureza_juridica,n
0,None,25032
1,SOCIEDADE EMPRESÁRIA LIMITADA,899
2,SOCIEDADE ANÔNIMA FECHADA,85
3,ASSOCIAÇÃO PRIVADA,76
4,SOCIEDADE EMPRESÁRIA,49
5,ENTIDADE FILANTRÓPICA,36
6,SOCIEDADE ANÔNIMA ABERTA,33
7,EMPRESÁRIO (INDIVIDUAL),24
8,SOCIEDADE DE ECONOMIA MISTA,21
9,SOCIEDADE SIMPLES PURA,12


## SQL modelo (para Streamlit com paginação)

In [9]:
SQL_TEMPLATE = (
"SELECT codigo_imovel, denominacao, municipio, uf, area_total_ha, titular, natureza_juridica, condicao_pessoa, percentual_detencao, pais "
"FROM imoveis "
"WHERE municipio_norm = ? AND area_total_ha BETWEEN ? AND ? AND titular_norm LIKE ? "
"ORDER BY area_total_ha DESC "
"LIMIT ? OFFSET ?;"
)